In [3]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import cv2

folder = 'fig'

In [11]:
src = cv2.imread(Path(folder, 'bamboo.jpg'))

if src is None:
    print('image read failed')
    sys.exit()

## blurring
kernel_3 = np.ones((3, 3), np.float32)/9.
dst = cv2.filter2D(src, -1, kernel_3)
dst_blur = cv2.blur(src, (3, 3))

## Gaussian
blur_sig1 = cv2.GaussianBlur(src, (0, 0), 1)
blur_sig2 = cv2.GaussianBlur(src, (0, 0), 2)

## sharpening
sharp_sig1 = cv2.addWeighted(src, 2, blur_sig1, -1, 0.0)
sharp_sig2 = cv2.addWeighted(src, 2, blur_sig2, -1, 0.0)

cv2.namedWindow('src', cv2.WINDOW_AUTOSIZE)
cv2.imshow('src', src)
# cv2.imshow('dst', dst)
# cv2.imshow('dst_blur', dst_blur)
# cv2.imshow('blur_sig1', blur_sig1)
# cv2.imshow('blur_sig2', blur_sig2)
cv2.imshow('sharp_sig1', sharp_sig1)
cv2.imshow('sharp_sig2', sharp_sig2)


while True:
    if cv2.waitKey() == 27:
        break
cv2.destroyAllWindows()

In [43]:
src = cv2.imread(Path(folder, 'spades.png'), cv2.IMREAD_REDUCED_COLOR_2)

## transform
affine = np.array([[1, 0, 100],
                   [0, 1, 50]], np.float32)
dst_trans = cv2.warpAffine(src, affine, (w+100, h+50))

## rotation
h, w = src.shape[:2]
rot_mat = cv2.getRotationMatrix2D((w//2, h//2), -20, 1.0)
print("rot matrix = \n", rot_mat)
dst_rot = cv2.warpAffine(src, rot_mat, (0, 0))

## 영상 확대
affine_scale = np.array([[1.2, 0, 0],
                         [0, 1.4, 0]], np.float32)
dst_scale = cv2.warpAffine(src, affine_scale, (int(w*1.2),int( h*1.4)))

## shear, skew, 전단 변환
affine_shear = np.array([[1, 0.2, 0],
                         [0, 1, 0]], np.float32)
dst_shear = cv2.warpAffine(src, affine_shear, (w+int(h*0.2), h))

cv2.imshow("src", src)
cv2.imshow("dst_trans", dst_trans)
cv2.imshow("dst_rot", dst_rot)
cv2.imshow("dst_scale", dst_scale)
cv2.imshow("dst_shear", dst_shear)

cv2.waitKey()
cv2.destroyAllWindows()

rot matrix = 
 [[  0.93969262  -0.34202014  51.49792289]
 [  0.34202014   0.93969262 -36.05923381]]


In [37]:
src = cv2.imread(Path(folder, 'son.jpg'))

dst1 = cv2.resize(src, (0, 0), fx=3, fy=3, interpolation=cv2.INTER_NEAREST)
dst2 = cv2.resize(src, (0, 0), fx=3, fy=3, interpolation=cv2.INTER_LINEAR)
dst3 = cv2.resize(src, (0, 0), fx=3, fy=3, interpolation=cv2.INTER_CUBIC)

cv2.imshow("src", src)
cv2.imshow("dst1", dst1)
cv2.imshow("dst2", dst2)
cv2.imshow("dst3", dst3)

cv2.waitKey()
cv2.destroyAllWindows()

# 비선형 변환 (Perspective transform)

In [44]:
# getPerspectiveTransform(src, dst[, solveMethod]) -> retval
# src: 입력영상의 4개 좌표점, numpy array shape(4,2)
# dst: 출력영상의 4개 좌표점, numpy array shape(4,2)

In [85]:
src = cv2.imread(Path(folder, 'checkerboard.png'))

# roi = cv2.selectROI(src)
# print('roi coor = ', roi)

src_lists = [[218, 49], [691, 47], [832, 516], [68, 527]]
h, w = src.shape[:2]

srcPoints = np.array(src_lists, np.float32)
dstPoints = np.array([[0, 0], [w-1, 0], [w-1, h-1], [0, h-1]], np.float32)

pers_mat = cv2.getPerspectiveTransform(srcPoints, dstPoints)

print("pers transforms matrix = \n", pers_mat)

dst = cv2.warpPerspective(src, pers_mat, (w, h))

cv2.imshow('src', src)
cv2.imshow('dst', dst)

cv2.waitKey(0)
cv2.destroyAllWindows()

pers transforms matrix = 
 [[ 2.13761247e+00  6.70798891e-01 -4.98868663e+02]
 [ 8.22754210e-03  1.94581371e+00 -9.71384758e+01]
 [-1.69199517e-05  1.37469054e-03  1.00000000e+00]]


# 객체 검출

In [94]:
img = cv2.imread(Path(folder, "bamboo.jpg"), cv2.IMREAD_GRAYSCALE)
img = cv2.imread(Path(folder, "plates.png"), cv2.IMREAD_GRAYSCALE)

sobel_kernelx = np.array([[-1, 0, 1],
                    [-2, 0, 2],
                    [-1, 0, 1]], np.float32)
sobel_kernely = sobel_kernelx.T

sobel_dx = cv2.filter2D(img, -1, sobel_kernelx)
sobel_dy = cv2.filter2D(img, -1, sobel_kernely)
# sobel_dx = cv2.absdiff(sobel_dx, )

cv2.imshow('img', img)
cv2.imshow('sobel_dx', sobel_dx)
cv2.imshow('sobel_dy', sobel_dy)

cv2.waitKey(0)
cv2.destroyAllWindows()

### Sobel filter

In [96]:
## Sobel filter
# cv2.Sobel(src, ddepth, dx, dy, dst, ksize, scale, delt, borderType) -> dst
# src : 입력영상
# ddepth : 출력영상의 데이터 타입 (-1)
# dx : x 방향 미분차수
# dy : x 방향 미분차수
# dst : 출력영상
# ksize : 커널의 크기
# scale : 연산결과에 추가적으로 곱할 값
# delta : 연산결과에 추가적으로 더할 값
# borderType : 가장자리 픽셀확장 방식

# magnitude(x, y, magnitude) -> magnitude

In [119]:
# src = cv2.imread(Path(folder, 'son.jpg'), cv2.IMREAD_GRAYSCALE)
src = cv2.imread(Path(folder, "plates.png"), cv2.IMREAD_GRAYSCALE)

dx = cv2.Sobel(src, cv2.CV_32F, 1, 0)
dy = cv2.Sobel(src, cv2.CV_32F, 0, 1)

## magnitude, direction
mag = cv2.magnitude(dx, dy)
mag = np.clip(mag, 0, 255).astype(np.uint8)

ret, dst = cv2.threshold(mag, 150, 255, cv2.THRESH_BINARY)

dst_canny = cv2.Canny(src, 150, 200)

cv2.imshow('src', src)
cv2.imshow('mag', mag)
cv2.imshow('dst', dst)
cv2.imshow('dst_canny', dst_canny)

cv2.waitKey(0)
cv2.destroyAllWindows()

### Hough transforms, 직선 검출

In [ ]:
## 직선 검출, 곡선 검출; 허프 변환 (Hough transform)
# HoughLinesP(image, rho, theta, threshold, lines, minLineLength, maxLineGap) -> lines
# image: 입력 에지영상
# rho: 축적배열에서 rho의 간격
# theta: 축적배열에서 theta의 간격
# threshold: 직선판단할 임계값
# lines: 선분의 끝좌표 (x1, y1, x2, y2)
# srn = None, stn = None
# minLineLength: 검출한 선분의 최소 길이
# maxLineGap: 직선으로 간주할 최대 에지 점 간격 (끝어진 점을 연결할 기준)

In [ ]:
# img = cv2.imread(Path(folder, 'checkerboard.png'))
img = cv2.imread(Path(folder, 'plates.png'))
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

edge = cv2.Canny(img_gray, 100, 200)

lines = cv2.HoughLinesP(edge, 1, np.pi/360, 69, minLineLength=20, maxLineGap=8)
print(lines.shape)

edge = cv2.cvtColor(edge, cv2.COLOR_GRAY2BGR)

for i in lines:
    pt1 = (i[0, 0], i[0, 1])
    pt2 = (i[0, 2], i[0, 3])
    cv2.line(edge, pt1, pt2, (0,255,0), 1, cv2.LINE_AA)

cv2.imshow('img', img)
cv2.imshow('img_gray', img_gray)
cv2.imshow('edge', edge)

cv2.waitKey()
cv2.destroyAllWindows()

(53, 1, 4)


In [169]:
# img = cv2.imread(Path(folder, 'checkerboard.png'))
# img = cv2.imread(Path(folder, 'plates.png'))
img = cv2.imread(Path(folder, 'door.png'), cv2.IMREAD_REDUCED_COLOR_2)
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

edge = cv2.Canny(img_gray, 100, 200)

lines = cv2.HoughLinesP(edge, 1, np.pi/360, 69, minLineLength=20, maxLineGap=8)
print(lines.shape)

edge = cv2.cvtColor(edge, cv2.COLOR_GRAY2BGR)

for i in lines:
    pt1 = (i[0, 0], i[0, 1])
    pt2 = (i[0, 2], i[0, 3])
    cv2.line(edge, pt1, pt2, (0,255,0), 1, cv2.LINE_AA)

cv2.imshow('img', img)
cv2.imshow('img_gray', img_gray)
cv2.imshow('edge', edge)

cv2.waitKey()
cv2.destroyAllWindows()

(40, 1, 4)


### Hough transform : 원 검출

In [170]:
# HoughCircles(image, method, dp, minDist[, circles[, param1[, param2[, minRadius[, maxRadius]]]]]) -> circles
# image: 입력영상
# method: cv2.HOUGH_GRADIENT,
# dp: 입력영상에 대한 실제 영상처리 배율, 1,  2 설정
# minDist: 검출된 원들간의 최소거리
# circles: 원좌표 (cx, cy, r), shape = (1, N, 3), dtype = np.float32
# param1: Canny edge max 값
# param2: 축척배열에서 원검출 임계값
# minRadius: 원 크기의 최소값
# maxRadius: 원 크기의 최대값

In [182]:
src = cv2.imread(Path(folder, 'plates.png'))
src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

circles = cv2.HoughCircles(src_gray, cv2.HOUGH_GRADIENT, 1, 50, param1=200, param2=50,
                           minRadius=50, maxRadius=150)

for i in circles[0]:
    cx, cy, r = i
    cv2.circle(src, (int(cx), int(cy)), int(r), (0, 255, 0), 2, cv2.LINE_AA)

cv2.imshow('src', src)
# cv2.imshow('src_gray', src_gray)

cv2.waitKey()
cv2.destroyAllWindows()

## 레이블링 (Labeling)

In [199]:
## 레이블링 (labeling)

# connectedComponentsWithStats(image[, labels[, stats[, centroids[, connectivity[, ltype]]]]]) -> retval, labels, stats, centroids

In [218]:
src = cv2.imread(Path(folder, 'symbols.png'))
# src = cv2.imread(Path(folder, 'shape.png'))

src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
ret, mask = cv2.threshold(src_gray, 150, 255, cv2.THRESH_BINARY_INV)

cnts, labels, stats, centroids = cv2.connectedComponentsWithStats(mask)

for i in range(1, cnts):
    (x, y, w, h, area) = stats[i]

    if area > 50:
        cv2.rectangle(src, (x, y, w, h), (0, 0, 255), 2)

# print('cnts = ', cnts)
# print('labels = ', labels)
# print('stats = ', stats)

cv2.imshow('src', src)
cv2.imshow('src_gray', src_gray)
cv2.imshow('mask', mask)

cv2.waitKey()
cv2.destroyAllWindows()

## 외곽선 검출

In [233]:
## 외곽선 검출
# findContours(image, mode, method[, contours[, hierarchy[, offset]]]) -> contours, hierarchy
    # image: 입력 영상. non-zero 픽셀을 객체로 간주함.
    # mode: 외곽선 검출 모드. cv2.RETR_로 시작하는 상수. 
    #     (cv2.RETR_EXTERNAL, cv2.RETR_LIST,cv2.RETR_CCOMP, cv2.RETR_TREE)
    # method: 외곽선 근사화 방법. cv2.CHAIN_APPROX_로 시작하는 상수.
    # contour: 검출된 외곽선 좌표. numpy.ndarray로 구성된 리스트. 
    # contours[i].shape=(K, 1, 2). contours[i].dtype=numpy.int32.
    # hierarchy: 외곽선 계층 정보. numpy.ndarray. shape=(1, N, 4). dtype=numpy.int32.
    # hierarchy[0, i, 0] ~ hierarchy[0, i, 3]이 순서대로 next, prev, child, parent
    # 외곽선 인덱스를 가리킴. 해당 외곽선이 없으면 -1.
    # offset: 좌표 값 이동 옵셋. 기본값은 (0, 0).
    
# drawContours(image, contours, contourIdx, color[, thickness[, lineType[, hierarchy[, maxLevel[, offset]]]]]) -> image

In [267]:
src = cv2.imread(Path(folder, 'plates.png'))
# src = cv2.imread(Path(folder, 'shape.png'))

src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
ret, mask = cv2.threshold(src_gray, 150, 255, cv2.THRESH_BINARY)

contours, hierarchy = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

for i in range(len(contours)):
    cv2.drawContours(src, contours, i, (0, 0, 255), 1, cv2.LINE_AA)
    cv2.putText(src, str(i), contours[i][0][0], 0, 0.6, (0,0,255), 1, cv2.LINE_AA)

cv2.imshow('src', src)
cv2.imshow('src_gray', src_gray)
cv2.imshow('mask', mask)

cv2.waitKey()
cv2.destroyAllWindows()

In [274]:
src = cv2.imread(Path(folder, 'img.jpg'))
# src = cv2.imread(Path(folder, 'shape.png'))

src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
ret, mask = cv2.threshold(src_gray, 150, 255, cv2.THRESH_BINARY_INV)

contours, hierarchy = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

for i in range(len(contours)):
    cv2.drawContours(src, contours, i, (0, 0, 255), 1, cv2.LINE_AA)
    cv2.putText(src, str(i), contours[i][0][0], 0, 0.6, (0,0,255), 1, cv2.LINE_AA)

cv2.imshow('src', src)
cv2.imshow('src_gray', src_gray)
cv2.imshow('mask', mask)

cv2.waitKey()
cv2.destroyAllWindows()

## 동영상

In [ ]:
cap = cv2.VideoCapture(Path(folder, 'PETS2000.avi'))

if not cap.isOpened():
    print('Video open failed')
    sys.exit()

while True:
    ret, frame = cap.read()
    # edge = cv2.Canny(frame, 100, 200)
    # gaussian = cv2.GaussianBlur(frame, (0, 0), 2)

    cv2.imshow('frame', frame)
    # cv2.imshow('edge', edge)
    # cv2.imshow('gaussian', gaussian)

    if cv2.waitKey(30) == 27:
        break

cv2.destroyAllWindows()
cap.release()